In [1]:
intcode = [3,8,1001,8,10,8,105,1,0,0,21,34,55,68,85,106,187,268,349,430,99999,3,9,1001,9,5,9,1002,9,5,9,4,9,99,3,9,1002,9,2,9,1001,9,2,9,1002,9,5,9,1001,9,2,9,4,9,99,3,9,101,3,9,9,102,3,9,9,4,9,99,3,9,1002,9,5,9,101,3,9,9,102,5,9,9,4,9,99,3,9,1002,9,4,9,1001,9,2,9,102,3,9,9,101,3,9,9,4,9,99,3,9,1001,9,2,9,4,9,3,9,101,2,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,1002,9,2,9,4,9,3,9,1001,9,1,9,4,9,3,9,1001,9,2,9,4,9,3,9,1002,9,2,9,4,9,3,9,101,2,9,9,4,9,3,9,1001,9,2,9,4,9,3,9,102,2,9,9,4,9,99,3,9,1002,9,2,9,4,9,3,9,102,2,9,9,4,9,3,9,101,1,9,9,4,9,3,9,1001,9,1,9,4,9,3,9,1001,9,1,9,4,9,3,9,101,1,9,9,4,9,3,9,1001,9,1,9,4,9,3,9,101,2,9,9,4,9,3,9,1001,9,1,9,4,9,3,9,1001,9,1,9,4,9,99,3,9,1001,9,2,9,4,9,3,9,101,2,9,9,4,9,3,9,101,2,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,102,2,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,101,1,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,102,2,9,9,4,9,3,9,1002,9,2,9,4,9,99,3,9,102,2,9,9,4,9,3,9,1001,9,2,9,4,9,3,9,102,2,9,9,4,9,3,9,102,2,9,9,4,9,3,9,101,1,9,9,4,9,3,9,102,2,9,9,4,9,3,9,102,2,9,9,4,9,3,9,102,2,9,9,4,9,3,9,101,1,9,9,4,9,3,9,1002,9,2,9,4,9,99,3,9,102,2,9,9,4,9,3,9,1001,9,1,9,4,9,3,9,102,2,9,9,4,9,3,9,101,1,9,9,4,9,3,9,102,2,9,9,4,9,3,9,102,2,9,9,4,9,3,9,101,2,9,9,4,9,3,9,101,2,9,9,4,9,3,9,1001,9,2,9,4,9,3,9,102,2,9,9,4,9,99]

In [2]:
from operator import add, mul

def compute_intcode(intcode, input_values=None):
    def read():
        mode3, mode2, mode1, code1, code2 = map(int, f'{intcode[pointer]:05}')
        return code1 * 10 + code2, mode1, mode2, mode3
    
    def getv(mode, shift):
        return intcode[pointer + shift] if mode else intcode[intcode[pointer + shift]]
    
    def setv(mode, value, shift=3):
        intcode[pointer + shift if mode else intcode[pointer + shift]] = value 
    
    results = []
    pointer = 0
    opcode, mode1, mode2, mode3 = read()
    while opcode != 99:
        if opcode == 1:
            setv(mode3, add(getv(mode1, 1), getv(mode2, 2)))
            pointer += 4
        elif opcode == 2:
            setv(mode3, mul(getv(mode1, 1), getv(mode2, 2)))
            pointer += 4
        elif opcode == 3:
            value = input_values.pop(0)
            setv(mode1, value, shift=1)
            pointer += 2
        elif opcode == 4:
            value = getv(mode1, 1)
            results.append(value)
            if value:
                input_value = (yield value)
                if input_value is not None:
                    input_values.append(input_value)
            pointer += 2
        elif opcode == 5:
            pointer = getv(mode2, 2) if getv(mode1, 1) else pointer + 3
        elif opcode == 6:
            pointer = getv(mode2, 2) if not getv(mode1, 1) else pointer + 3
        elif opcode == 7:
            setv(mode3, 1) if getv(mode1, 1) < getv(mode2, 2) else setv(mode3, 0)
            pointer += 4
        elif opcode == 8:
            setv(mode3, 1) if getv(mode1, 1) == getv(mode2, 2) else setv(mode3, 0)
            pointer += 4
        opcode, mode1, mode2, mode3 = read()
    return results[-1]

In [3]:
def amplify(intcode, sequence=None, input_value=0):
    sequence = sequence or []
    if not sequence:
        return None
    input_sequence = sequence.pop(0)
    output_value = next(compute_intcode(intcode[:], input_values=[input_sequence, input_value]))
    if sequence:
        return amplify(intcode, sequence=sequence, input_value=output_value)
    else:
        return output_value

In [4]:
from itertools import permutations

max_output = 0
for sequence in permutations(range(5), 5):
    output_value = amplify(intcode[:], sequence=list(sequence))
    max_output = max(max_output, output_value)
max_output

255840

In [5]:
from itertools import cycle

def amplify_with_feedback(intcode, sequence=None, input_value=0):
    sequence = sequence or []
    if not sequence:
        return None
    amplifiers, outputs = {}, []
    for i, seq in enumerate(cycle(sequence)):
        current = i % len(sequence)
        previous = len(sequence) - 1 if current - 1 < 0 else current - 1
        amplifier = amplifiers.setdefault(current, compute_intcode(
            intcode[:], input_values=[seq, outputs[-1] if outputs else input_value]))
        try:
            if i < len(sequence):
                output = next(amplifier)
            else:
                output = amplifier.send(outputs[-1])
        except StopIteration:
            return outputs[-1]
        outputs.append(output)

In [6]:
max_output = 0
for sequence in permutations(range(5,10), 5):
    try:
        output_value = amplify_with_feedback(intcode, sequence=list(sequence))
        max_output = max(max_output, output_value)
    except Exception as e:
        continue
max_output

84088865